In [1]:
import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json

/Users/test/Library/Caches/pypoetry/virtualenvs/transport-frames-QNnroJIm-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_json_file(filename):
    with open(filename, 'r') as f:
        json_object = json.load(f)

    result_dict = {}

    for key in json_object.keys():
        if isinstance(json_object[key], str):  # check if the value is a string
            try:
                gdf = gpd.read_file(json_object[key])
                result_dict[key] = gdf
            except:
                try:
                    result_dict[key] = json.loads(json_object[key])
                except json.JSONDecodeError:
                    result_dict[key] = json_object[key]
        else:
            result_dict[key] = json_object[key]

    return result_dict


In [4]:
res = read_json_file('/Users/test/Documents/code/tf_api/app/db/indicators/Волгоградская_область/Волгоградская_область_indicators_gpsp.json')

# Inputs

In [10]:
from pprint import pprint

pprint(res.keys())

dict_keys(['aggregated_density', 'road_length_gdf', 'connectivity', 'connectivity_public_transport', 'to_fed_roads', 'to_region_admin_center', 'azs_availability', 'number_of_fuel_stations', 'train_stops_availability', 'number_of_train_stops', 'international_aero_availability', 'number_of_international_aero', 'local_aero_availability', 'number_of_local_aero', 'number_of_bus_stops', 'train_paths_length', 'number_of_bus_routes'])


In [19]:
res['azs_availability'].explore(column='to_service')

In [8]:
res['aggregated_density']

,id,name,layer,density,geometry
0,0,Революционное,Палласовский район,30.011,"MULTIPOLYGON (((46.78903 49.63836, 46.73639 49..."
1,1,Стеженское,Алексеевский район,720.273,"MULTIPOLYGON (((41.97687 50.29195, 41.97739 50..."
2,2,Краснооктябрьское,Алексеевский район,455.375,"MULTIPOLYGON (((42.36857 50.28091, 42.36860 50..."
3,3,Ларинское,Алексеевский район,819.841,"MULTIPOLYGON (((42.21293 50.28499, 42.21300 50..."
4,4,Яминское,Алексеевский район,1466.216,"MULTIPOLYGON (((42.19721 50.32547, 42.20512 50..."
...,...,...,...,...,...
429,429,Красноярское,Чернышковский район,1775.423,"MULTIPOLYGON (((42.27296 48.40632, 42.27870 48..."
430,430,Большетерновское,Чернышковский район,2238.868,"MULTIPOLYGON (((42.29665 48.45521, 42.32227 48..."
431,431,Пристеновское,Чернышковский район,1145.534,"MULTIPOLYGON (((42.72587 48.28015, 42.72692 48..."
432,432,Алешкинское,Чернышковский район,1042.120,"MULTIPOLYGON (((42.48190 48.22617, 42.49394 48..."


In [16]:
def create_json_file(d1, filename):
    json_dict = {}
    for key in d1.keys():
        if isinstance(d1[key],(float,int)):
            json_dict[key] = d1[key]
        else:
            json_dict[key] = d1[key].to_json()
    json_object = json.dumps(json_dict)
    with open(filename, 'w') as f:
        f.write(json_object)

In [17]:
# Ленинградская_область EPSG: 32636  / 176095 / N27025179 (Гатчина)
# Санкт_Петербург EPSG:32636 / 337422 / N27490597
# Москва EPSG:32637 / 102269 / N1686293227
# Волгоградская_область EPSG:32638 / 77665 /  N27504363
# Тульская_область EPSG:32637 /  81993 / N34389350
# Омская_область EPSG:32643 / 140292 /  N27503946
# Краснодарский_край EPSG:32637 / 108082 / N27505129
# Тюменская_область EPSG:32642 / 140291 / N27505666
# Московская_область EPSG:32637 / 51490 / N1686293227 

In [18]:
region_name = 'Тульская_область'
local_crs = 32637
geocode = 81993
region_capital = ox.geocode_to_gdf('N34389350',by_osmid=True)

In [19]:
city = ox.geocode_to_gdf(f'R{geocode}', by_osmid=True).to_crs(epsg=4326)
city['name'] = region_name

In [21]:

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # set where are no services 

polygons_settlement = gpd.read_file(f'../data/{region_name}/{region_name}_settlement.geojson')
polygons_district = gpd.read_file(f'../data/{region_name}/{region_name}_district.geojson')

points = gpd.read_file(f'../data/{region_name}/{region_name}_region_points.geojson')
points_admin_centres_settlements = shapely.centroid(polygons_settlement.geometry).set_crs(polygons_settlement.crs)
points_admin_centres_districts = points[points['is_admin_centre_district']==True]


railway_stations = gpd.read_file(f'../data/{region_name}/{region_name}_railway_stations.geojson')

fuel_stations = gpd.read_file(f'../data/{region_name}/{region_name}_fuel_stations.geojson')

ferry_terminal = PLACEHOLDER # для некоторых регионов есть gpd.read_file(f'../data/{region_name}/{region_name}_ferry_terminal.geojson')

aero = gpd.read_file(f'../data/{region_name}/{region_name}_aerodrome.geojson')

international_aero = aero[aero['aerodrome:type']=='international']
international_aero['geometry'] = shapely.centroid(international_aero['geometry']).set_crs(international_aero.crs)

aero = aero[aero['aerodrome:type']!='international']
aero['geometry'] = shapely.centroid(aero['geometry']).set_crs(aero.crs)


water_objects = gpd.read_file(f'../data/{region_name}/{region_name}_water.geojson')

oopt = gpd.read_file(f'../data/{region_name}/{region_name}_nature_reserve.geojson')

train_paths = gpd.read_file(f'../data/{region_name}/{region_name}_railway_roads.geojson')

bus_stops = gpd.read_file(f'../data/{region_name}/{region_name}_bus_stop.geojson')

russia = ox.geocode_to_gdf("Russia") 
regions = gpd.read_file(f'../data/regions_of_russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

# test territory
neud = polygons188.iloc[[0]] # для ЛО лучше подавать по одному файлу (принимает однострочный гдф) - сиверский и тд и тд тд

# Preprocessing

In [22]:
import os

directory = f"../preprocessed/{region_name}/"
os.makedirs(directory, exist_ok=True)

In [23]:
# write an uds graph
citygraph = graphbuilder.get_graph_from_polygon(city, crs=local_crs,country_polygon=russia)
citygraph_ = graphbuilder.convert_list_attr_to_str(citygraph)
nx.write_graphml(citygraph_, f"../preprocessed/{region_name}/{region_name}_uds.graphml")

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 13 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [24]:
# # write inter into graphml
inter = indicators.get_intermodal(geocode,local_crs)
e = [
        (u, v, k)
        for u, v, k, d in inter.edges(data=True, keys=True)
        if d.get("type") in (['bus','walk'])
    ]
inter = inter.edge_subgraph(e)

# save routes and stops
ni,ei = momepy.nx_to_gdf(inter)
bus_routes = ei[ei['type']=='bus']
bus_routes.to_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# convert to file
inter_= graphbuilder.convert_list_attr_to_str(inter)
for node,e,data in inter_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
nx.write_graphml(inter_, f"../preprocessed/{region_name}/{region_name}_inter.graphml")

2024-06-14 20:48:34.520 | INFO     | dongraphio.dongraphio:get_intermodal_graph_from_osm:61 - Creating intermodal graph from OSM...
2024-06-14 20:48:34.521 | WARNING  | dongraphio.base_models.grapher_logic:get_public_trasport_graph:237 - Files with routes or with stops was not found. The graph will be built based on data from OSM
2024-06-14 20:48:35.596 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:162 - It seems there are no subway routes in the city. This transport type will be skipped.
2024-06-14 20:48:50.117 | INFO     | dongraphio.base_models.grapher_logic:get_public_trasport_graph:285 - Public transport graph done!
2024-06-14 20:48:50.693 | DEBUG    | dongraphio.base_models.grapher_logic:get_osmnx_graph:95 - Extracting and preparing walk graph from OSM ...
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 12 times your configured Overpass max query area size. It will automa

In [25]:
# write frame into graphml
frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
frame = graphbuilder.assign_city_names_to_nodes(points18.reset_index().to_crs(frame.graph['crs']), 
                                                momepy.nx_to_gdf(frame)[0], frame, name_col='name', max_distance=1200)
frame_ = frame.copy()
for node,e,data in frame_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
for node,data in frame_.nodes(data=True):
    if 'ref' in data:
        data['ref']=str(data['ref'])
frame_ = graphbuilder.convert_list_attr_to_str(frame_)
nx.write_graphml(frame_, f"../preprocessed/{region_name}/{region_name}_frame.graphml")

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_59551/1013225357.py:2: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:505: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n["nodeID"] == path[j]), "weight"] += weight
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:506: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08]' has dtype incompatible with int64, please explicitly cast to a c

# Processing

In [26]:
citygraph = nx.read_graphml(f'../preprocessed/{region_name}/{region_name}_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [27]:
inter = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_inter.graphml")
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)

In [28]:
bus_routes = gpd.read_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# # or if we don't have them:
# ni,ei = momepy.nx_to_gdf(inter)
# bus_routes = ei[ei['type']=='bus']

In [ ]:
# показатели для территории +3 км (подается однострочный гдф в качестве территории)
d3 = indicators.indicator_territory(citygraph, neud,
                                    regions_gdf=polygons_district,
                                    districts_gdf=polygons_settlement,
                                    region_capital=region_capital,
                                    region_centers=points_admin_centres_districts,
                                    district_centers=polygons_settlement,
                                    settlement_centers=points,
                                    fuel=fuel_stations,
                                    train_stops=railway_stations,
                                    international_aero=international_aero,
                                    aero=aero,
                                    ports=ferry_terminal,
                                    water_objects=water_objects, 
                                    oopt=oopt, 
                                    train_paths=train_paths, 
                                    inter = None,
                                    bus_stops=bus_stops, 
                                    bus_routes=bus_routes,crs=local_crs)

2024-06-14 20:34:33.616 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:41.735 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 20:34:41.936 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 20:34:41.941 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:49.356 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 20:34:49.550 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 20:34:49.566 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:59.954 | DEBUG    | dongraphio.base_models.matrix

In [ ]:
n_grade = grade_territory.grade_territory(neud, frame, include_priority=True) # оценка территории (любой в границах региона) относительно транспортного каркаса, возвращает чиселку с оценкой

In [ ]:
n_criteria = criteria.get_criteria(neud,points,polygons_settlement,inter) # вычисление критерия территории для карточки, возвращает гдф с чиселками

# end